In [5]:
from sage.all import *
from sage.misc.prandom import sample, shuffle, random

load("alpha_od.sage")

# --------------------------------------------------
# strong odd independent set
# --------------------------------------------------
def is_strong_odd_IS(G, S):

    if len(S) % 2 == 0:
        return False

    for u in S:
        for v in S:
            if u != v and G.has_edge(u, v):
                return False

    for v in G.vertices():
        if v not in S:
            cnt = sum(1 for u in S if G.has_edge(u, v))
            if cnt % 2 == 0:
                return False

    return True


# --------------------------------------------------
# local triangle check
# --------------------------------------------------
def local_triangle_check(G, verts):
    for v in verts:
        Nv = set(G.neighbors(v))
        for u in Nv:
            if Nv.intersection(G.neighbors(u)):
                return True
    return False


# --------------------------------------------------
# exact property (i)
# --------------------------------------------------
def satisfies_i(G):

    if not G.is_connected():
        return False

    if G.diameter() > 3:
        return False

    verts = G.vertices()
    r = G.degree(verts[0])

    if r % 2 == 0:
        return False
    if any(G.degree(v) != r for v in verts):
        return False

    alpha = alpha_od_ilp_correct(G)
    if alpha != r:
        return False

    for v in verts:
        if not is_strong_odd_IS(G, set(G.neighbors(v))):
            return False

    return True


# --------------------------------------------------
# heuristic scoring
# --------------------------------------------------
def heuristic_score(G):

    score = 0

    # many strong odd neighborhoods
    for v in G.vertices():
        if is_strong_odd_IS(G, set(G.neighbors(v))):
            score += 3

    # prefer small diameter
    try:
        d = G.diameter()
        score += max(0, 4 - d)
    except:
        pass

    # lightweight alpha check
    if score > 0:
        try:
            score += min(5, alpha_od_ilp(G))
        except:
            pass

    return score


# --------------------------------------------------
# degree preserving swap
# --------------------------------------------------
def random_2switch(G, trials=40):

    edges = list(G.edges(labels=False))
    if len(edges) < 2:
        return None

    for _ in range(trials):

        (a,b),(c,d) = sample(edges,2)
        if len({a,b,c,d})<4:
            continue

        swaps=[((a,c),(b,d)),((a,d),(b,c))]
        shuffle(swaps)

        for (x,y),(u,v) in swaps:

            if G.has_edge(x,y) or G.has_edge(u,v):
                continue

            H = G.copy()
            H.delete_edge(a,b)
            H.delete_edge(c,d)
            H.add_edge(x,y)
            H.add_edge(u,v)

            if not H.is_connected():
                continue

            if local_triangle_check(H,[a,b,c,d,x,y,u,v]):
                continue

            return H

    return None


# --------------------------------------------------
# simulated annealing walk
# --------------------------------------------------
def annealing_walk(G0, steps=5000, T0=5.0, cool=0.999):

    G = G0.copy()
    best = G.copy()

    current_score = heuristic_score(G)
    best_score = current_score

    T = T0

    for step in range(steps):

        H = random_2switch(G)
        if H is None:
            continue

        new_score = heuristic_score(H)
        delta = new_score - current_score

        if delta >= 0 or random() < exp(delta / max(T,0.001)):
            G = H
            current_score = new_score

        if new_score > best_score:
            best = H.copy()
            best_score = new_score

        T *= cool

        # reheating
        if step % 1500 == 0 and step>0:
            T = T0

    return best


# --------------------------------------------------
# Task 2 full search
# --------------------------------------------------
def task2_research_search(G0, restarts=20, steps=5000):

    if not G0.is_regular():
        raise ValueError("Initial graph must be r-regular")

    found = []
    seen = set()

    for r in range(restarts):

        print(f"\n=== restart {r} ===")

        G = annealing_walk(G0, steps=steps)

        key = G.canonical_label().graph6_string()
        if key in seen:
            continue
        seen.add(key)

        if satisfies_i(G):
            print("FOUND graph satisfying (i)")
            found.append(G.copy())

            # ----- PRIKAZ GRAFA -----
            show(G)  # odpre graf v oknu Sage / Jupyter

            # ----- SHRANJEVALNJE KOT PNG -----
            fname_png = f"graph_{len(found)}.png"
            G.show(fname_png)
            print(f"  Graf shranjen kot {fname_png}")

            # ----- SHRANJEVALNJE KOT .graph6 -----
            fname_g6 = f"graph_{len(found)}.graph6"
            with open(fname_g6, 'w') as f:
                f.write(key)
            print(f"  Graf shranjen kot {fname_g6}")

    return found


In [8]:

G0 = graphs.PetersenGraph()

results = task2_research_search(G0, restarts=300, steps=6000)

print("Najdenih grafov:", len(results))




=== restart 0 ===


FOUND graph satisfying (i)


  Graf shranjen kot graph_1.png
  Graf shranjen kot graph_1.graph6

=== restart 1 ===



=== restart 2 ===



=== restart 3 ===


...


=== restart 206 ===



=== restart 207 ===



=== restart 208 ===



=== restart 209 ===



=== restart 210 ===
